In [ ]:
import multiprocessing as mp
import scipy

In [ ]:
def work(start_weights, sigma):
    global nn, env
    scipy.random.seed()
    candidate = params_perturbation(start_weights, sigma=sigma)
    modified_params = combine_weights(start_weights, candidate)
    nn.set_params(modified_params)
    reward = evaluate_model(nn, env, verbose=False)
    return candidate, reward

In [ ]:
def generation_update_mp(model, environment, pool, sigma=0.1, lr=0.01, population_size=10, seed=None, normalize_rewards=True):
    original_params = deep_copy(model.l)
    
    if seed is not None:
        np.random.seed(seed)

    population = []
    rewards = []
    tasks = [(original_params, sigma) for i in range(population_size)]
    results = pool.starmap(work, tasks)
    population = [p[0] for p in results]
    rewards = [p[1] for p in results]

    rewards = np.array(rewards)
    r_mean, r_std = rewards.mean(), rewards.std()
    if normalize_rewards:
        rewards = (rewards - r_mean) / (r_std + 1e-9)
        
    update_params(original_params, population, rewards, learning_rate=lr, sigma=sigma)
    model.set_params(deep_copy(original_params))
    return r_mean, r_std

In [ ]:
pool = mp.Pool(mp.cpu_count())